&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; <b>YOLO Loss Function</b>

<img src="https://i.stack.imgur.com/bQAdd.jpg">

### Set up

In [1]:
import numpy as np
import pandas as pd
import keras.backend as K

Using TensorFlow backend.


In [2]:
img_height = 448
img_width = 448
S = 7
B = 2
C = 2
grid_cell_h = img_height/S
grid_cell_w = img_width/S

In [3]:
a = 0
b = 0
grid_x = []
grid_y = []
for i in range(S):
    grid_x.append([a, a+img_height/S])
    grid_y.append([b, b+img_width/S])
    a += img_height/S
    b += img_width/S

In [4]:
y_pred = np.random.rand(S, S, (5*B+C))
y_true = np.zeros((S, S, 5*B+C))
y_true[6, 0, :5] = np.random.rand(5,)
y_true[6, 0, -C:] = np.random.rand(C,)
y_true[1, 6, :] = np.random.rand(5*B+C,)
y_true[4, 2, :] = np.random.rand(5*B+C,)
# y_pred[6, 0, :] = np.random.rand(7,)
# y_pred[0, 6, :] = np.random.rand(7,)

<pre>third dimension of y_true and y_pred: 5*B+C
 - last C values are for classes
 - order of each Box values: x, y, w, h, C 
</pre>

### Experiments

In [25]:
i = 0
t = (np.sum(y_true[:, :, i*5:(i+1)*5], axis=2)>0).astype(int)
stacked = np.stack([t for ind in range(5)], axis=2)
stacked.shape

np.sum(stacked, axis=2)

(7, 7, 5)

In [45]:

I_1  = np.zeros((S,S,5*B+C), dtype='uint8')# i,j
for i in range(B):
    t = (np.sum(y_true[:, :, i*5:(i+1)*5], axis=2)>0).astype(int)
    stacked = np.stack([t for ind in range(5)], axis=2)
    I_1[:, :, i*5:(i+1)*5] = stacked

(I_1*y_true).shape

(7, 7, 12)

In [46]:
I_1_noobj = np.logical_not(I_1).astype(int)
I_1_noobj[:, :, -C:] = np.zeros((S,S,2))

I_1_noobj.shape

(7, 7, 12)

In [55]:
a = y_true[:, :, -C+1:]
b = (np.sum(a, axis=2)>0).astype(int)
I_2 = np.stack([b for ind in range(C)], axis=2)

In [40]:
B = 2
# y_true = np.zeros((S, S, 5*B))
C_ = list(range(4, 5*B+1, 5))
print(C_) 
I__ = y_true[:, :, C_]
I__.shape

C_ = np.array([4])
C_ = [j+i*5 for i in range(B) for j in C_]
C_

[4, 9]


[4, 9]

In [120]:
x_ = np.array([0,1])
x__ = []
for i in range(B):
    x__.extend(x_+i*5)
    
# y_true[:, :, B:2]
x__
y_true[:, :, x__].shape
a_ = y_pred[:, :, x__]
b_ = y_true[:, :, x__]
np.sum(((a_-b_)**2)*I_1)

0.73057063989319271

In [25]:
#one liner
x__1 = [j+i*5 for i in range(B) for j in x_]
# print([(i, j) for i in range(5) for j in range(5,10)]) # to understand

In [121]:
w_ = np.array([2,3])
w__ = []
for i in range(B):
    w__.extend(w_+i*5)
# y_true[:, :, B:2]
w__
# y_true[:, :, w__].shape
a_ = np.sqrt(y_pred[:, :, w__])
b_ = np.sqrt(y_true[:, :, w__])
np.sum(((a_-b_)**2)*I_1)

0.059362076506496629

In [122]:
c_ = np.array([4])
c__ = []
for i in range(B):
    c__.extend(c_+i*5)
# y_true[:, :, B:2]
c__
a_ = y_true[:, :, c__]
b_ = y_pred[:, :, c__]
np.sum(((a_-b_)**2)*I_1)

0.73079649517707657

In [125]:
a_ = y_true[:, :, -C:]
b_ = y_pred[:, :, -C:]
np.sum(((a_-b_)**2)*I_2)

0.8497298726897492

### Final loss in Numpy

In [7]:
lamdba_1 = 5
lambda_2 = 0.05

In [71]:
def loss(y_pred, y_true):
    '''getting the first term'''
    I_1  = np.zeros((S,S,5*B+C), dtype='uint8')# i,j
    for i in range(B):
        t = (np.sum(y_true[:, :, i*5:(i+1)*5], axis=2)>0).astype(int)
        stacked = np.stack([t for ind in range(5)], axis=2)
        I_1[:, :, i*5:(i+1)*5] = stacked

    x_ = np.array([0,1])
    x__ = [j+i*5 for i in range(B) for j in x_]    
    a_ = y_pred[:, :, x__]*I_1[:, :, x__]
    b_ = y_true[:, :, x__]*I_1[:, :, x__]

    term1 = lamdba_1*np.sum((a_-b_)**2)
   
    
    '''second term'''
    w_ = np.array([2,3])
    w__ = [j+i*5 for i in range(B) for j in w_]
    c_ = np.sqrt(y_pred[:, :, w__])*I_1[:, :, w__]
    d_ = np.sqrt(y_true[:, :, w__])*I_1[:, :, w__]

    term2 = lamdba_1*np.sum((c_-d_)**2)
    
    
    '''third term'''
    C_ = np.array([4])
    C_ = [j+i*5 for i in range(B) for j in C_]
    e_ = y_pred[:, :, C_]*I_1[:, :, C_]
    f_ = y_true[:, :, C_]*I_1[:, :, C_]
    term3 = np.sum((e_-f_)**2)
    
    '''fourth term'''
    I_1_noobj = np.logical_not(I_1).astype(int)
    I_1_noobj[:, :, -C:] = np.zeros((S,S,2))

    g_ = y_pred[:, :, C_]*I_1_noobj[:, :, C_]
    h_ = y_true[:, :, C_]*I_1_noobj[:, :, C_]
    term4 = lambda_2*np.sum((g_-h_)**2)

    '''fifth term'''
    
    a = y_true[:, :, -C+1:] # 0th class is no object
    b = (np.sum(a, axis=2)>0).astype(int)
    I_2 = np.stack([b for ind in range(C)], axis=2)
    
    i_ = y_pred[:, :, -C:]*I_2
    j_ = y_true[:, :, -C:]*I_2
#     print(i_.shape, j_.shape)
    term5 = np.sum((i_-j_)**2)
    
    
    total_loss = term1 + term2 + term3 + term4 + term5
    return total_loss

In [72]:
loss(y_pred=y_pred, y_true=y_true)

10.91543715656109

### Second Experimentation

In [4]:
import tensorflow as tf
import keras.backend as K
vec = K.tf.constant([1, 2, 3, 4])
multiply = K.tf.constant([3])

K.eval(K.reshape(K.tf.tile(vec, multiply), shape=(multiply[0], tf.shape(vec)[0])))

array([[1, 2, 3, 4],
       [1, 2, 3, 4],
       [1, 2, 3, 4]], dtype=int32)

## Final loss in Keras / tf

In [20]:
y_true_ = K.variable(y_true)
y_pred_ = K.variable(y_pred)

In [ ]:
def custom_loss(self, y_true, y_pred):
    mask_shape = tf.shape(y_true)[:4]

    cell_x = tf.to_float(tf.reshape(tf.tile(tf.range(self.grid_w), [self.grid_h]), (1, self.grid_h, self.grid_w, 1, 1)))
    cell_y = tf.transpose(cell_x, (0,2,1,3,4))

    cell_grid = tf.tile(tf.concat([cell_x,cell_y], -1), [self.batch_size, 1, 1, self.nb_box, 1])

    coord_mask = tf.zeros(mask_shape)
    conf_mask  = tf.zeros(mask_shape)
    class_mask = tf.zeros(mask_shape)

    seen = tf.Variable(0.)
    total_recall = tf.Variable(0.)

    """
    Adjust prediction
    """
    ### adjust x and y      
    pred_box_xy = tf.sigmoid(y_pred[..., :2]) + cell_grid

    ### adjust w and h
    pred_box_wh = tf.exp(y_pred[..., 2:4]) * np.reshape(self.anchors, [1,1,1,self.nb_box,2])

    ### adjust confidence
    pred_box_conf = tf.sigmoid(y_pred[..., 4])

    ### adjust class probabilities
    pred_box_class = y_pred[..., 5:]

    """
    Adjust ground truth
    """
    ### adjust x and y
    true_box_xy = y_true[..., 0:2] # relative position to the containing cell

    ### adjust w and h
    true_box_wh = y_true[..., 2:4] # number of cells accross, horizontally and vertically

    ### adjust confidence
    true_wh_half = true_box_wh / 2.
    true_mins    = true_box_xy - true_wh_half
    true_maxes   = true_box_xy + true_wh_half

    pred_wh_half = pred_box_wh / 2.
    pred_mins    = pred_box_xy - pred_wh_half
    pred_maxes   = pred_box_xy + pred_wh_half       

    intersect_mins  = tf.maximum(pred_mins,  true_mins)
    intersect_maxes = tf.minimum(pred_maxes, true_maxes)
    intersect_wh    = tf.maximum(intersect_maxes - intersect_mins, 0.)
    intersect_areas = intersect_wh[..., 0] * intersect_wh[..., 1]

    true_areas = true_box_wh[..., 0] * true_box_wh[..., 1]
    pred_areas = pred_box_wh[..., 0] * pred_box_wh[..., 1]

    union_areas = pred_areas + true_areas - intersect_areas
    iou_scores  = tf.truediv(intersect_areas, union_areas)

    true_box_conf = iou_scores * y_true[..., 4]

    ### adjust class probabilities
    true_box_class = tf.argmax(y_true[..., 5:], -1)

    """
    Determine the masks
    """
    ### coordinate mask: simply the position of the ground truth boxes (the predictors)
    coord_mask = tf.expand_dims(y_true[..., 4], axis=-1) * self.coord_scale

    ### confidence mask: penelize predictors + penalize boxes with low IOU
    # penalize the confidence of the boxes, which have IOU with some ground truth box < 0.6
    true_xy = self.true_boxes[..., 0:2]
    true_wh = self.true_boxes[..., 2:4]

    true_wh_half = true_wh / 2.
    true_mins    = true_xy - true_wh_half
    true_maxes   = true_xy + true_wh_half

    pred_xy = tf.expand_dims(pred_box_xy, 4)
    pred_wh = tf.expand_dims(pred_box_wh, 4)

    pred_wh_half = pred_wh / 2.
    pred_mins    = pred_xy - pred_wh_half
    pred_maxes   = pred_xy + pred_wh_half    

    intersect_mins  = tf.maximum(pred_mins,  true_mins)
    intersect_maxes = tf.minimum(pred_maxes, true_maxes)
    intersect_wh    = tf.maximum(intersect_maxes - intersect_mins, 0.)
    intersect_areas = intersect_wh[..., 0] * intersect_wh[..., 1]

    true_areas = true_wh[..., 0] * true_wh[..., 1]
    pred_areas = pred_wh[..., 0] * pred_wh[..., 1]

    union_areas = pred_areas + true_areas - intersect_areas
    iou_scores  = tf.truediv(intersect_areas, union_areas)

    best_ious = tf.reduce_max(iou_scores, axis=4)
    conf_mask = conf_mask + tf.to_float(best_ious < 0.6) * (1 - y_true[..., 4]) * self.no_object_scale

    # penalize the confidence of the boxes, which are reponsible for corresponding ground truth box
    conf_mask = conf_mask + y_true[..., 4] * self.object_scale

    ### class mask: simply the position of the ground truth boxes (the predictors)
    class_mask = y_true[..., 4] * tf.gather(self.class_wt, true_box_class) * self.class_scale       

    """
    Warm-up training
    """
    no_boxes_mask = tf.to_float(coord_mask < self.coord_scale/2.)
    seen = tf.assign_add(seen, 1.)

    true_box_xy, true_box_wh, coord_mask = tf.cond(tf.less(seen, self.warmup_bs), 
                          lambda: [true_box_xy + (0.5 + cell_grid) * no_boxes_mask, 
                                   true_box_wh + tf.ones_like(true_box_wh) * np.reshape(self.anchors, [1,1,1,self.nb_box,2]) * no_boxes_mask, 
                                   tf.ones_like(coord_mask)],
                          lambda: [true_box_xy, 
                                   true_box_wh,
                                   coord_mask])

    """
    Finalize the loss
    """
    nb_coord_box = tf.reduce_sum(tf.to_float(coord_mask > 0.0))
    nb_conf_box  = tf.reduce_sum(tf.to_float(conf_mask  > 0.0))
    nb_class_box = tf.reduce_sum(tf.to_float(class_mask > 0.0))

    loss_xy    = tf.reduce_sum(tf.square(true_box_xy-pred_box_xy)     * coord_mask) / (nb_coord_box + 1e-6) / 2.
    loss_wh    = tf.reduce_sum(tf.square(true_box_wh-pred_box_wh)     * coord_mask) / (nb_coord_box + 1e-6) / 2.
    loss_conf  = tf.reduce_sum(tf.square(true_box_conf-pred_box_conf) * conf_mask)  / (nb_conf_box  + 1e-6) / 2.
    loss_class = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=true_box_class, logits=pred_box_class)
    loss_class = tf.reduce_sum(loss_class * class_mask) / (nb_class_box + 1e-6)

    loss = loss_xy + loss_wh + loss_conf + loss_class

    if self.debug:
        nb_true_box = tf.reduce_sum(y_true[..., 4])
        nb_pred_box = tf.reduce_sum(tf.to_float(true_box_conf > 0.5) * tf.to_float(pred_box_conf > 0.3))

        current_recall = nb_pred_box/(nb_true_box + 1e-6)
        total_recall = tf.assign_add(total_recall, current_recall) 

        loss = tf.Print(loss, [tf.zeros((1))], message='Dummy Line \t', summarize=1000)
        loss = tf.Print(loss, [loss_xy], message='Loss XY \t', summarize=1000)
        loss = tf.Print(loss, [loss_wh], message='Loss WH \t', summarize=1000)
        loss = tf.Print(loss, [loss_conf], message='Loss Conf \t', summarize=1000)
        loss = tf.Print(loss, [loss_class], message='Loss Class \t', summarize=1000)
        loss = tf.Print(loss, [loss], message='Total Loss \t', summarize=1000)
        loss = tf.Print(loss, [current_recall], message='Current Recall \t', summarize=1000)
        loss = tf.Print(loss, [total_recall/seen], message='Average Recall \t', summarize=1000)

    return loss

In [23]:
def loss_K():
    pass